In [1]:
import pandas as pd
import os 
import re

In [2]:
def ft_field_transformation(field_name):
    field_name = re.sub("(_{2})", '_', field_name)
    # matches = re.finditer('.+?(?:(?<=[a-z])(?=[A-Z])|(?<=[A-Z])(?=[A-Z][a-z])|$)', field_name)
    matches = re.finditer('.+?(?:(?<=[a-z])(?=[A-Z])|'         # Small letter following by a capital letter
                           # '(?<=[A-Z])(?=[A-Z][a-z])|'       # Capital letter following by 1-Capital letter and then 2- Small letter Note: at this time fivetran does not support this   
                          '(?<=[0-9])(?=([a-z])|([A-Z]))|'     # Digit following by a letter either capital or small 
                          '(?<=([a-z])|([A-Z]))(?=[0-9])|'     # A letter following by digit 
                          '$)', field_name)
    string_in_list = [m.group(0) for m in matches]
    return '_'.join(string_in_list).lower()

In [34]:
def bi_fsm_join(bi_df,fsm_df):
    fsm_df.rename(columns = {"Field Name": "fsm_field_name", "Table Name": "fsm_table_name"},inplace = True)
    fsm_df['bi_field_name'] = fsm_df['fsm_field_name'].apply(ft_field_transformation) 
    fsm_df['bi_table_name'] = fsm_df['fsm_table_name'].apply(ft_field_transformation) 
    bi_df.rename(columns = {'Type':'bi_dataflow_type'},inplace = True)
    merge_dfs = pd.merge(bi_df,fsm_df,left_on=['Field Name','Table Name'], right_on = ['bi_field_name','bi_table_name'],how = 'left')
    return merge_dfs 

In [35]:
fsm_df = pd.read_csv('C:\\Users\\moe.salari\\Data Dictionary - Account.csv')
bi_df = pd.read_csv('C:\\Users\\moe.salari\\Book1.csv')